In [5]:
import sys
!{sys.executable} -m pip install --upgrade --no-cache-dir \
    numpy==1.26.4 pandas==2.1.4 \
    dash==2.* jupyter-dash dash-leaflet==0.* dash-bootstrap-components==1.* plotly==5.* \
    "pymongo>=4.6,<5" backports.tarfile


Defaulting to user installation because normal site-packages is not writeable


In [6]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html, dcc, dash_table, Input, Output
import dash_leaflet as dl
import dash_bootstrap_components as dbc

from animal_shelter import AnimalShelter

shelter = AnimalShelter(
    user="aacuser",
    pwd="uT9~kV3-Pw1_Lz7.qR5nY2sB8",   
    host="nv-desktop-services.apporto.com",
    port=32508,
    db="AAC",
    col="animals",
)

PROJECTION = {
    "_id": 0,
    "name": 1,
    "breed": 1,
    "animal_type": 1,
    "sex_upon_outcome": 1,
    "age_upon_outcome_in_weeks": 1,
    "outcome_type": 1,
    "outcome_subtype": 1,
    "location_lat": 1,
    "location_long": 1,
}


In [7]:
RESCUE_QUERIES = {
    "Water": {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
    },
    "Mountain/Wilderness": {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
    },
    "Disaster/Tracking": {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
    },
}

def fetch_df(filter_key=None):
    q = RESCUE_QUERIES.get(filter_key, {})
    rows = shelter.read(q, projection=PROJECTION)
    df = pd.DataFrame(rows)
    if df.empty:
        return df
    df["location_lat"]  = pd.to_numeric(df["location_lat"], errors="coerce")
    df["location_long"] = pd.to_numeric(df["location_long"], errors="coerce")
    df = df.dropna(subset=["location_lat","location_long"]).reset_index(drop=True)
    return df.rename(columns={
        "name": "Name",
        "breed": "Breed",
        "animal_type": "Type",
        "sex_upon_outcome": "Sex",
        "age_upon_outcome_in_weeks": "Age (wks)",
        "outcome_type": "Outcome Type",
        "outcome_subtype": "Outcome Subtype",
        "location_lat": "Lat",
        "location_long": "Lon",
    })

df_all = fetch_df()
if df_all.empty:
    df_all = pd.DataFrame(columns=["Name","Breed","Type","Sex","Age (wks)","Outcome Type","Outcome Subtype","Lat","Lon"])



In [8]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])


logo_src = "/assets/logo.png"   
branding = dbc.Row([
    dbc.Col(html.A(html.Img(src=logo_src, style={"height":"70px"}), href="https://www.snhu.edu", target="_blank"), md=2),
    dbc.Col([html.H3("Grazioso Salvare — Search & Rescue Dashboard"),
             html.Small("jonbosch1199")], md=10, style={"textAlign":"right"}),
], align="center")

controls = dbc.Card(dbc.CardBody([
    dcc.RadioItems(
        id="filter-radio",
        options=[
            {"label":"Reset (All)", "value":"Reset"},
            {"label":"Water Rescue", "value":"Water"},
            {"label":"Mountain/Wilderness", "value":"Mountain/Wilderness"},
            {"label":"Disaster/Tracking", "value":"Disaster/Tracking"},
        ],
        value="Reset",
        labelStyle={"display":"block"}
    )
]))

table = dash_table.DataTable(
    id="datatable",
    data=df_all.to_dict("records"),
    columns=[{"name":c,"id":c} for c in df_all.columns],
    page_size=10,
    filter_action="native",
    sort_action="native",
    sort_mode="multi",
    row_selectable="single",
    selected_rows=[0],
    style_table={"height":"500px","overflowY":"auto"},
    style_cell={"textAlign":"left","whiteSpace":"normal"},
)

the_map = dl.Map(id="map", style={"width":"100%","height":"500px"},
                 center=[30.75,-97.48], zoom=9, children=[dl.TileLayer(id="base")])

chart = dcc.Graph(id="pie")

app.layout = dbc.Container([
    branding, html.Hr(),
    dbc.Row([dbc.Col(controls, md=3), dbc.Col(table, md=9)], align="start"),
    html.Br(),
    dbc.Row([dbc.Col(the_map, md=6), dbc.Col(chart, md=6)]),
], fluid=True)


/home/jonathanbosch_snhu/.local/lib/python3.9/site-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [9]:
@app.callback(
    Output("datatable","data"),
    Output("pie","figure"),
    Input("filter-radio","value"),
)
def apply_filter(filter_value):
    dff = df_all.copy() if filter_value=="Reset" else fetch_df(filter_value)
    if dff.empty:
        return [], px.scatter(title="No data")
    fig = px.pie(dff, names="Breed", title="Breed mix")
    return dff.to_dict("records"), fig

@app.callback(
    Output("map","children"),
    Input("datatable","derived_virtual_data"),
    Input("datatable","derived_virtual_selected_rows"),
)
def update_map(view_rows, sel_rows):
    dff = pd.DataFrame(view_rows) if view_rows else df_all.copy()
    if dff.empty:
        return [dl.TileLayer(id="base")]
    row = sel_rows[0] if sel_rows else 0
    lat = float(dff.iloc[row]["Lat"]); lon = float(dff.iloc[row]["Lon"])
    breed = str(dff.iloc[row]["Breed"]); name = str(dff.iloc[row]["Name"])
    return [dl.TileLayer(id="base"),
            dl.Marker(position=[lat, lon],
                      children=[dl.Tooltip(breed), dl.Popup([html.H5("Animal"), html.P(name)])])]

@app.callback(
    Output("datatable","style_data_conditional"),
    Input("datatable","derived_virtual_selected_rows"),
)
def style_selected(sel_rows):
    return [] if not sel_rows else [{"if":{"row_index":sel_rows[0]},"backgroundColor":"#FFF3CD"}]



In [10]:
app.run_server(mode="inline", port=8054, debug=False)



ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8054): Max retries exceeded with url: /_alive_c4ebbb7b-13bd-4510-8ed9-0697d32d745b (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f461f5548e0>: Failed to establish a new connection: [Errno 111] Connection refused'))